# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Dual model with Clarabel attached

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       ga

The lower bound found is 3

In [7]:
solution_summary(model)

* Solver : Dual model with Clarabel attached

* Status
  Result count       : 1
  Termination status : ALMOST_OPTIMAL
  Message from the solver:
  "ALMOST_SOLVED"

* Candidate solution (result #1)
  Primal status      : NEARLY_FEASIBLE_POINT
  Dual status        : NEARLY_FEASIBLE_POINT
  Objective value    : 3.00000e+00
  Dual objective value : 3.00000e+00

* Work counters
  Solve time (sec)   : 3.80250e-01
  Barrier iterations : 25


The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [L09, (6.15)].

[L09] Laurent, Monique.
*Sums of squares, moment matrices and optimization over polynomials.*
Emerging applications of algebraic geometry (2009): 157-270.

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  1.0000000017942992     -0.999989143710501      …   1.7053053568112501e-7
 -0.999989143710501       0.9999783243534897         6.797989423469114e-8
  1.580102593106209e-5   -1.5771972745812904e-5      2.79696701840826e-7
  0.9999783072999893     -0.9999674674920328         4.647622407969194e-5
 -1.5786666526493362e-5   1.5803029613779264e-5      4.876404311575872e-5
  1.1323687290922323e-7  -3.750689669898932e-8   …   9.450720417176097e-5
 -0.9999674117632373      0.9999566722211841        -0.0006788961387083221
  1.5800448377331057e-5  -1.579995109911538e-5      -0.0010861136843415642
  7.342911073028707e-9    1.4323043649043317e-7     -0.001443038296573526
  8.20862258382363e-8     6.51535261135398e-8       -0.0021945096989402974
  0.9999566700632712

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = measure(ν, atol = 1e-5)

Measure{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}([1.0000000017942992, -0.999989143710501, 1.580102593106209e-5, 0.9999783072999893, -1.5786666526493362e-5, 1.1323687290922323e-7, -0.9999674117632373, 1.5800448377331057e-5, 7.342911073028707e-9, 8.20862258382363e-8  …  -0.0021945096989402974, 1.5012396326670379, -0.3664568576392168, 0.5672190390117505, -0.26573151069000334, 0.715987679666811, -0.0410781652118392, 1.0505421700738642, 0.46168908097995054, 1.8018743040423588], DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³  …  x₁⁷, x₂⁸, x₁

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  1.0000000017942992     -0.999989143710501      …   8.20862258382363e-8
 -0.999989143710501       0.9999783072999893         6.801985355837143e-8
  1.580102593106209e-5   -1.5786666526493362e-5      1.7053053568112501e-7
  0.9999783072999893     -0.9999674117632373        -5.194381992715694e-9
 -1.5786666526493362e-5   1.5800448377331057e-5      6.797989423469114e-8
  1.1323687290922323e-7   7.342911073028707e-9   …   2.79696701840826e-7
 -0.9999674117632373      0.9999566700632712         1.7057184431936546e-5
  1.5800448377331057e-5  -1.578574352355621e-5       4.647622407969194e-5
  7.342911073028707e-9    7.323462890186348e-8       4.876404311575872e-5
  8.20862258382363e-8     6.801985355837143e-8       9.450720417176097e-5

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 3.9998738488272467
 0.00014858202106434175
 2.9538170321778437e-5
 3.9445311450139666e-7
 1.298822750570064e-7
 7.383992070844883e-8
 7.137853776215193e-8
 5.6548076457455015e-8
 2.2012043333595885e-8
 3.365360038791725e-9

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.196427254487122
 2.7012010863685187
 1.738742316554198
 0.00038475922885237426
 0.00010800491691338999
 2.7579831986545214e-5
 2.3799442037633358e-5
 4.5451574756198407e-7
 1.4168137174886528e-7
 2.9359880377630318e-8
 1.0751169947369091e-8
 1.4669490520969313e-9
 5.433360224336174e-10
 2.399183381207747e-11
 1.8344496494568505e-11

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [1.581694412093348e-5, -0.9999891631593412] with weight 1.033433351536792

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*